In [1]:
import os
from datetime import timedelta, datetime, date

import polars as pl
from supabase import create_client
from google.cloud import storage, secretmanager

from data_wrangling import load_season, record_current_season
from elo_rating import elo_season
from modelling import lgbm_model
from data_collection import (
    season,
    collect_season_statistics,
    collect_season_filtered_table,
    collect_all_data,
    collect_season_data
)

/Users/nicohahn/Documents/nba-game-classification/nba/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../cloud-key.json"
secret_client = secretmanager.SecretManagerServiceClient()
response = secret_client.access_secret_version(request={'name':'projects/898760610238/secrets/supabase/versions/1'})
creds = eval(response.payload.data.decode("UTF-8"))
connection = create_client(creds['postgres']['project_url'], creds['postgres']['api_key'])

season_dates = pl.DataFrame(connection.table('season').select('*').execute().data).with_columns([
    pl.col('all_star_date').str.to_date(),
    pl.col('play_in_start').str.to_date(),
    pl.col('play_in_end').str.to_date()
])
res = connection.table('schedule').select('date, season_id').order('date', desc=True).limit(1).execute().data[0]
newest_date = datetime.strptime(res['date'], '%Y-%m-%d').date()
season_id = res['season_id']

In [7]:
print('Collecting Schedule and Boxscores')
season(
    start_date=newest_date,
    end_date=newest_date + timedelta(days=3),
    connection=connection,
    season_id=season_id
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [28:29<00:00, 106.84s/it]


In [28]:
print('Collecting Season Statistics')
season_2024 = load_season(
    season_dates.filter(pl.col('season_id') == season_id)['all_star_date'][0],
    season_dates.filter(pl.col('season_id') == season_id)['play_in_start'][0],
    season_dates.filter(pl.col('season_id') == season_id)['play_in_end'][0],
    connection=connection,
    season_id=season_id
)
previous_df, recent_games_df, remainder_df, season_df = collect_season_statistics(season_id, connection)
new_data_1 = season_2024.filter(~pl.col('game_id').is_in(previous_df['game_id'])).select(previous_df.columns)
new_data_2 = season_2024.filter(~pl.col('game_id').is_in(recent_games_df['game_id'])).select(recent_games_df.columns)
new_data_3 = season_2024.filter(~pl.col('game_id').is_in(remainder_df['game_id'])).select(remainder_df.columns)
new_data_4 = season_2024.filter(~pl.col('game_id').is_in(season_df['game_id'])).select(season_df.columns)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:01<00:00, 29.62it/s]


In [29]:
update_date = remainder_df[["game_id", "is_home_win"]].join(
    season_2024[["game_id", "date", "is_home_win"]],
    on="game_id"
).filter(
    pl.col("is_home_win").is_null() & (pl.col("is_home_win_right").is_not_null())
)["date"].min()
update_data_1 = season_2024.filter(
    (pl.col("date") >= update_date) & (~pl.col("game_id").is_in(new_data_1["game_id"]))
).drop_nulls(
    ["fieldGoalsMade_previous_game_home_team", "fieldGoalsMade_previous_game_away_team"]
).select(previous_df.columns)
update_data_2 = season_2024.filter(
    (pl.col("date") >= update_date) & (~pl.col("game_id").is_in(new_data_2["game_id"]))
).select(recent_games_df.columns)
update_data_3 = season_2024.filter(
    (pl.col("date") >= update_date) & (~pl.col("game_id").is_in(new_data_3["game_id"]))
).select(remainder_df.columns)
update_data_4 = season_2024.filter(
    (pl.col("date") >= update_date) & (~pl.col("game_id").is_in(new_data_4["game_id"]))
).drop_nulls(
   ["fieldGoalsMade_109_home_team", "fieldGoalsMade_109_away_team"]
).select(season_df.columns)

In [31]:
if new_data_1.shape[0] > 0:
    response = (
        connection.table('statistics_previous').insert(
            new_data_1.to_dicts()
        ).execute()
    )
if new_data_2.shape[0] > 0:
    response = (
        connection.table('statistics_recent_games').insert(
            new_data_2.to_dicts()
        ).execute()
    )
if new_data_3.shape[0] > 0:
    response = (
        connection.table('statistics_remainder').insert(
            new_data_3.to_dicts()
        ).execute()
    )
if new_data_4.shape[0] > 0:
    response = (
        connection.table('statistics_season').insert(
            new_data_4.to_dicts()
        ).execute()
    )

if update_data_1.shape[0] > 0:
    response = (
        connection.table('statistics_previous').upsert(
            update_data_1.to_dicts()
        ).execute()
    )
if update_data_2.shape[0] > 0:
    response = (
        connection.table('statistics_recent_games').upsert(
            update_data_2.to_dicts()
        ).execute()
    )
if update_data_3.shape[0] > 0:
    response = (
        connection.table('statistics_remainder').upsert(
            update_data_3.to_dicts()
        ).execute()
    )
if update_data_4.shape[0] > 0:
    response = (
        connection.table('statistics_season').upsert(
            update_data_4.to_dicts()
        ).execute()
    )

In [32]:
h2h_current_year = load_season(
    season_dates.filter(pl.col('season_id') == season_id)['all_star_date'][0],
    season_dates.filter(pl.col('season_id') == season_id)['play_in_start'][0],
    season_dates.filter(pl.col('season_id') == season_id)['play_in_end'][0],
    connection=connection,
    season_id=season_id,
    return_h2h=True
)
h2h_supabase = collect_season_filtered_table(season_id, 'h2h', connection)
new_data_5  = h2h_current_year.filter(~pl.col('game_id').is_in(h2h_supabase['game_id'])).to_dicts()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:01<00:00, 29.67it/s]


In [35]:
if len(new_data_5) > 0:
    response = (
        connection.table('h2h').insert(
            new_data_5
        ).execute()
    )
rec_current_year = record_current_season(
    season_dates.filter(pl.col('season_id') == season_id)['all_star_date'][0],
    season_dates.filter(pl.col('season_id') == season_id)['play_in_start'][0],
    season_dates.filter(pl.col('season_id') == season_id)['play_in_end'][0],
    connection=connection,
    season_id=season_id,
).drop_nulls('games_this_year_home_team')
rec_current_year_supabase = collect_season_filtered_table(season_id, 'record', connection)
schedule = collect_season_data(season_id, 'schedule', connection)
new_data_6 = rec_current_year.filter(
    ~pl.col('game_id').is_in(rec_current_year_supabase['game_id'])
).join(
    schedule[["game_id", "date"]],
    on="game_id"
).filter(
    pl.col("date").str.to_date() <= date.today()
).drop("date").to_dicts()

In [ ]:
if len(new_data_6) > 0:
    response = (
        connection.table('record').insert(
            new_data_6
        ).execute()
    )
update_data_6 = rec_current_year.join(
    rec_current_year_supabase[["game_id", "points_home"]],
    on="game_id"
).filter(
    pl.col("points_home").is_not_null() & pl.col("points_home_right").is_null()
).drop("points_home_right").to_dicts()
if len(update_data_6) > 0:
    response = (
        connection.table('record').upsert(
            update_data_6
        ).execute()
    )

In [42]:
df_list = []
for s_id in season_dates['season_id']:
    if s_id == season_id:
        df_list.append(rec_current_year)
    else:
        df_list.append(
            collect_season_filtered_table(s_id, 'record', connection)
        )
df_list = [
    rec_current_year if s_id == season_id else collect_season_filtered_table(s_id, 'record', connection)
    for s_id in season_dates['season_id']
]
schedule_df = collect_all_data('schedule', connection)
df_list = [df.join(schedule_df, on='game_id').drop('season_id') for df in df_list]
df_list[-1] = df_list[-1][df_list[0].columns]
elo_df_list = []
for i in range(len(df_list)):
    if i == 0:
        elo_df_list.append(
            elo_season(df_list[i])
        )
    else:
        elo_df_list.append(
            elo_season(df_list[i].drop_nulls('points_home'), elo_df_list[i - 1])
        )
elo_df = pl.concat(elo_df_list)
elo_df_supabase = collect_all_data('elo', connection)
new_data_7 = elo_df.join(
    elo_df_supabase[['game_id', 'team_id', 'elo_before']],
    on=['game_id', 'team_id'],
    how='left'
).filter(pl.col('elo_before_right').is_null()).drop('elo_before_right').to_dicts()

In [44]:
if len(new_data_7) > 0:
    response = (
        connection.table('elo').insert(
            new_data_7
        ).execute()
    )

In [45]:
client = storage.Client()
bucket = client.get_bucket("lgbm")
mod = lgbm_model(
    connection = connection,
    bucket = bucket,
    data_origin="supabase"
)
mod.load_model()

In [54]:
mod.predict()

Downloaded newest matching file: lgbm_evaluation_february_2025.json to lgbm_evaluation_february_2025.json
7 predictions added


In [48]:
best_features, cutoff_date = self.load_best_features()
previous_predictions = collect_all_data('predictions', self.connection)

Downloaded newest matching file: lgbm_evaluation_february_2025.json to lgbm_evaluation_february_2025.json


In [49]:
cutoff_date = self.full_data.join(
    previous_predictions,
    on='game_id',
    how='inner'
)['date'].max()
X_new = self.full_data.to_dummies([
    'game_type', 'month', 'weekday'
]).filter(
    (pl.col('date') >= cutoff_date) &
    (pl.col('is_home_win').is_not_null()) &
    (~pl.col('game_id').is_in(previous_predictions['game_id']))
)

datetime.date(2025, 3, 2)

In [53]:
game_ids = X_new['game_id']
X_new = X_new.select(best_features).drop('date')
predictions = self.model.predict(X_new.to_numpy())
prediction_df = pl.DataFrame({
    'game_id': game_ids,
    'probability': predictions,
    'is_home_win': predictions >= 0.5
})
prediction_df

game_id,probability,is_home_win
str,f64,bool
"""0022400881""",0.022659,false
"""0022400879""",0.447127,false
"""0022400877""",0.994281,true
"""0022400876""",0.304303,false
"""0022400875""",0.185216,false
"""0022400880""",0.05036,false
"""0022400878""",0.982777,true
